<a href="https://colab.research.google.com/github/shiv827/Pytorch-Tutorial/blob/main/02_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np

In [ ]:
# Inputs (temp, rainfall, humidity)

inputs = np.array([[73, 67, 43],[91, 88, 64], [87, 134, 58], [102, 43, 37], 
                  [69, 96, 70]], dtype='float32')

In [ ]:
inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [ ]:
# Targets (yeild of apple, yeild of oranges)

targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119]]
                  , dtype = 'float32')
targets

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

In [ ]:
# Convert the numpy arrays to pytorch tensors

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [ ]:
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [ ]:
# Weights and biases
torch.manual_seed(42)
w = torch.randn(2,3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]], requires_grad=True)
tensor([ 2.2082, -0.6380], requires_grad=True)


In [ ]:
def model(x):
    return x @ w.t() + b

In [ ]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  45.4987,  -67.0672],
        [  59.1878,  -90.4141],
        [  62.3595, -141.8688],
        [  50.7645,  -32.3210],
        [  54.2179, -105.5822]], grad_fn=<AddBackward0>)


In [ ]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [ ]:
# Improving the model with MSE loss function
def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff) / diff.numel()

"""
Adjust weights and biases using gradient descent to reduce the gradient
 improve the model. Follow the following steps:
1. Generate predictions
2. Calculate loss
3. Compute gradient wrt weights and biases
3. Adjust the weights by subtracting a small uantity proportional to the gradient
5. Reset the gradient to zero
"""


In [ ]:
loss = mse(preds, targets)
print(loss)

tensor(19322.3477, grad_fn=<DivBackward0>)


In [ ]:
loss.backward()

In [ ]:
print(w.grad)
print(b.grad)

tensor([[ -1622.2405,  -2731.8169,  -1496.6278],
        [-14781.0176, -17480.2129, -10474.4824]])
tensor([ -21.7943, -179.4507])


In [ ]:
# Train the model for 100 epochs

for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w-=w.grad * 1e-5
        b-=b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
# Calculate the loss after training
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(376.1279, grad_fn=<DivBackward0>)


Linear regression using built in packages


In [ ]:
import torch.nn
import numpy as np

In [ ]:
# Take inputs (temp, rainfall, humidity)

inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                   [102, 43, 37], [69, 96, 70], [74, 66, 43],
                   [91, 87, 65], [88, 134, 59], [101, 44, 37],
                   [68, 96, 71], [73, 66, 44], [92, 87, 64], 
                   [87, 135, 57], [103, 43, 36], [68, 97, 70]], dtype='float32')



In [ ]:
inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.],
       [ 74.,  66.,  43.],
       [ 91.,  87.,  65.],
       [ 88., 134.,  59.],
       [101.,  44.,  37.],
       [ 68.,  96.,  71.],
       [ 73.,  66.,  44.],
       [ 92.,  87.,  64.],
       [ 87., 135.,  57.],
       [103.,  43.,  36.],
       [ 68.,  97.,  70.]], dtype=float32)

In [ ]:
# Targets as (yeild of apples, yeild of oranges)
targets = np.array([[56, 70], [81, 101], [119, 133],
                    [22, 37], [103, 119], [57, 69],
                    [80, 102], [118, 132], [21, 38],
                    [104, 118], [57, 69], [82, 100],
                    [118, 134], [20, 38], [102, 120]], dtype = 'float32')

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

Dataset and Dataloader
We will create a TensorDataset which allows access to rows from inputs and targets as tuples and provised stadard APIs from working with many different types of datasets in pytorch 

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
# Define dataset

train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [ ]:
# Define dataloader
from torch.utils.data import DataLoader

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
# We can use a DataLoader in a for loop

for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 91.,  87.,  65.],
        [ 73.,  67.,  43.],
        [ 88., 134.,  59.]])
tensor([[ 81., 101.],
        [119., 133.],
        [ 80., 102.],
        [ 56.,  70.],
        [118., 132.]])


In [ ]:
import torch.nn

In [ ]:
# Model Building

model = torch.nn.Linear(3,2) 
# 3 inputs and 2 outputs
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5278,  0.2960, -0.0714],
        [-0.5090, -0.4731, -0.4345]], requires_grad=True)
Parameter containing:
tensor([-0.1734, -0.2811], requires_grad=True)


In [ ]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.5278,  0.2960, -0.0714],
         [-0.5090, -0.4731, -0.4345]], requires_grad=True),
 Parameter containing:
 tensor([-0.1734, -0.2811], requires_grad=True)]

In [ ]:
# Generate predictions

preds = model(inputs)

In [ ]:
preds

tensor([[  55.1125,  -87.8185],
        [  69.3277, -116.0398],
        [  81.2604, -133.1574],
        [  63.7427,  -88.6195],
        [  59.6561, -111.2330],
        [  55.3443,  -87.8544],
        [  68.9603, -116.0013],
        [  81.7168, -134.1009],
        [  63.5109,  -88.5835],
        [  59.0569, -111.1585],
        [  54.7451,  -87.7799],
        [  69.5595, -116.0758],
        [  81.6278, -133.1959],
        [  64.3419,  -88.6940],
        [  59.4244, -111.1970]], grad_fn=<AddmmBackward0>)

In [ ]:
# Loss Function
import torch.nn.functional as F

In [ ]:
loss_fn = F.mse_loss

In [ ]:
loss = loss_fn(preds, targets)
print(loss)

tensor(21699.7598, grad_fn=<MseLossBackward0>)


In [ ]:
# Define Optimizer

opt = torch.optim.SGD(model.parameters(), lr = 1e-5)


In [ ]:
# Train the model


1. Generate Prediction
2. Calculate the loss
3. Compute gradient wrt weights and biases
4. Adjust the weights by subtracting a smalll quantity proportional to the gradient
5. Reset the gradeint to zero

In [ ]:
# Utility funtion to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

  # Repeat the given no of epochs
  for epoch in range(num_epochs):
    
    # Train with batches of data
    for xb, yb in train_dl:

      # 1. Generate prediction
      preds = model(xb)

      # 2. Calculate loss
      loss = loss_fn(preds, yb)

      # 3. Compute gradient
      loss.backward()

      # 4. Update parameters using gradients
      opt.step()

      # 5. Reset the gradient to zero
      opt.zero_grad()

    # Print the progress
    if (epoch+1) % 10 ==0:
      print('Epoch[{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


In [ ]:
fit(100, model, loss_fn, opt, train_dl)

Epoch[10/100], Loss: 249.3945
Epoch[20/100], Loss: 307.3629
Epoch[30/100], Loss: 396.4355
Epoch[40/100], Loss: 168.9442
Epoch[50/100], Loss: 87.2535
Epoch[60/100], Loss: 93.9222
Epoch[70/100], Loss: 72.5715
Epoch[80/100], Loss: 72.8188
Epoch[90/100], Loss: 49.6780
Epoch[100/100], Loss: 43.6305


In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [110., 132.],
        [ 22.,  37.],
        [130., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [ ]:
preds = model(inputs)

In [ ]:
preds

tensor([[ 58.6152,  71.6113],
        [ 79.5044,  97.7030],
        [121.4414, 137.3112],
        [ 30.1572,  44.7009],
        [ 91.9650, 109.4590],
        [ 57.5191,  70.6126],
        [ 78.7957,  97.2099],
        [121.4866, 137.6335],
        [ 31.2532,  45.6996],
        [ 92.3523, 109.9645],
        [ 57.9065,  71.1182],
        [ 78.4084,  96.7044],
        [122.1501, 137.8043],
        [ 29.7698,  44.1953],
        [ 93.0610, 110.4577]], grad_fn=<AddmmBackward0>)

In [ ]:
# Testing on a data point

model(torch.tensor([[75, 63, 44.]]))

tensor([[54.7894, 68.2137]], grad_fn=<AddmmBackward0>)

In [ ]:
# The predicted yeild of apples is 54.7 tons per hectare and that of orangaes is 
# 68 tons per hectare.